In [1]:
import MDAnalysis as md
import numpy as np
import os
import re

In [2]:
pwd

'/data1/lisbeth/SPC/CG_SEP/BOUND'

In [83]:
#AA_complex_bound_dir = '/data1/lisbeth/SPC/CG_NEW/PRODUCTION/BOUND/'
CG_complex_bound_dir = '/data1/lisbeth/SPC/CG_SEP/BOUND/'

os.chdir(AA_complex_bound_dir)
u = md.Universe('PDB/SPC_L11_AA_1_memb_prot.pdb','XTC/SPC_L11_AA_1_whole_fit.xtc')

In [23]:
os.chdir(CG_complex_bound_dir)

#### Load in the coarse grained simulation given a tpr/gro/pdb and the corresponding xtc file 

In [70]:
## Relative long simulation, can take time to load in
ucg = md.Universe('TPR/SPC_L11_CG_2.tpr', 'XTC/Before_Corrections/SPC_L11_CG_1_whole.xtc')

In [75]:
number_of_frames = ucg.trajectory.n_frames

In [76]:
number_of_frames #Enough frames? 

1000001

### To iterate over a trajectory (either atomistic or coarse-grained, dosent matter here) :

In [ ]:
for ts in ucg.trajectory:
    print ("DO STUFF")
    lipid = ucg.select_atoms('resname POPC') #Make an atom selection
    coor = lipid.positions #Get X,Y,Z coordinates for the atoms given the current frame

#### The trajectory can be splitted like so:

In [ ]:
for ts in ucg.trajectory[:number_of_frames/2]:
    print ("DO STUFF")

In [ ]:
for ts in ucg.trajectory[number_of_frames/2:]:
    print ("DO STUFF")

## Essentially it works like a numpy array you can index/stride

###### Functions

In [72]:
def get_dict_types(sel):
    '''Given an atom selection from MDAnalysis, 
    it will output a dictionary contaning all the beadtypes
    with their corresponding vdw radii.'''
    D = {}
    for idx, i in enumerate(sel.types):
        name = sel.names[idx]
        if re.match('^T', i):
            radii = 0.191
        elif re.match('^S', i):
            radii = 0.230
        else:
            radii = 0.264
        D[i] = radii
    return D

def get_dict_names(sel):
    '''Given an atom selection from MDAnalysis, 
    it will output a dictionary contaning all the unique beadnames
    with their corresponding vdw radii.
    NB! This is not great, if the same beadname is used with different
    beadtypes, hence different vdw radii'''
    D = {}
    for idx, i in enumerate(sel.types):
        name = sel.names[idx]
        if re.match('^T', i):
            radii = 0.191
        elif re.match('^S', i):
            radii = 0.230
        else:
            radii = 0.264
        D[name] = radii
    return D

## Generating dictionary for the protein residues. For each resname you will have a dictionary with the beadnames and corresponding vdw radii

In [34]:
prot = ucg.select_atoms('name BB SC1 SC2 SC3 SC4 SC5')

In [64]:
Dict = {}
for res in np.unique(prot.resnames):
    R = prot.select_atoms('resname {}'.format(res))
    Dict[res] = get_dict_names(R)

In [67]:
Dict

{'ALA': {'BB': 0.23, 'SC1': 0.191},
 'ARG': {'BB': 0.264, 'SC1': 0.23, 'SC2': 0.23},
 'ASN': {'BB': 0.264, 'SC1': 0.23},
 'ASP': {'BB': 0.264, 'SC1': 0.23},
 'CYS': {'BB': 0.264, 'SC1': 0.191},
 'GLN': {'BB': 0.264, 'SC1': 0.264},
 'GLU': {'BB': 0.264, 'SC1': 0.264},
 'GLY': {'BB': 0.264},
 'HIS': {'BB': 0.264, 'SC1': 0.191, 'SC2': 0.191, 'SC3': 0.191},
 'ILE': {'BB': 0.264, 'SC1': 0.23},
 'LEU': {'BB': 0.264, 'SC1': 0.23},
 'LYS': {'BB': 0.264, 'SC1': 0.23, 'SC2': 0.23},
 'MET': {'BB': 0.264, 'SC1': 0.264},
 'PHE': {'BB': 0.264, 'SC1': 0.23, 'SC2': 0.191, 'SC3': 0.191},
 'PRO': {'BB': 0.23, 'SC1': 0.23},
 'SER': {'BB': 0.264, 'SC1': 0.191},
 'THR': {'BB': 0.264, 'SC1': 0.23},
 'TRP': {'BB': 0.264,
  'SC1': 0.191,
  'SC2': 0.191,
  'SC3': 0.191,
  'SC4': 0.191,
  'SC5': 0.191},
 'TYR': {'BB': 0.264, 'SC1': 0.191, 'SC2': 0.191, 'SC3': 0.191, 'SC4': 0.191},
 'VAL': {'BB': 0.23, 'SC1': 0.23}}

#### Example for a lipid, given you have loaded a tpr file instead of a gro or pdb

In [73]:
lipid = ucg.select_atoms('resname POPC')

In [77]:
get_dict_types(lipid)

{'Q1': 0.264,
 'Q5': 0.264,
 'SN4a': 0.23,
 'N4a': 0.264,
 'C1': 0.264,
 'C4h': 0.264}

#### If you want to generate a dict of the entire system, protein, lipid, small molecules:

In [78]:
sel = ucg.select_atoms('all')

In [80]:
np.unique(sel.resnames)

array(['ALA', 'ARG', 'ASN', 'ASP', 'CHOL', 'CYS', 'GLN', 'GLU', 'GLY',
       'HIS', 'ILE', 'ION', 'LEU', 'LYS', 'MET', 'PHE', 'POPC', 'POPE',
       'POPS', 'PRO', 'SAP2', 'SER', 'THR', 'TRP', 'TYR', 'VAL', 'W'],
      dtype=object)

In [81]:
Dict_all = {}
for res in np.unique(sel.resnames):
    s = sel.select_atoms('resname {}'.format(res))
    Dict_all[res] = get_dict_names(s)

In [82]:
Dict_all

{'ALA': {'BB': 0.23, 'SC1': 0.191},
 'ARG': {'BB': 0.264, 'SC1': 0.23, 'SC2': 0.23},
 'ASN': {'BB': 0.264, 'SC1': 0.23},
 'ASP': {'BB': 0.264, 'SC1': 0.23},
 'CHOL': {'ROH': 0.23,
  'R1': 0.23,
  'R2': 0.23,
  'R3': 0.23,
  'R4': 0.23,
  'R5': 0.23,
  'C1': 0.23,
  'C2': 0.264},
 'CYS': {'BB': 0.264, 'SC1': 0.191},
 'GLN': {'BB': 0.264, 'SC1': 0.264},
 'GLU': {'BB': 0.264, 'SC1': 0.264},
 'GLY': {'BB': 0.264},
 'HIS': {'BB': 0.264, 'SC1': 0.191, 'SC2': 0.191, 'SC3': 0.191},
 'ILE': {'BB': 0.264, 'SC1': 0.23},
 'ION': {'NA': 0.191, 'CL': 0.191},
 'LEU': {'BB': 0.264, 'SC1': 0.23},
 'LYS': {'BB': 0.264, 'SC1': 0.23, 'SC2': 0.23},
 'MET': {'BB': 0.264, 'SC1': 0.264},
 'PHE': {'BB': 0.264, 'SC1': 0.23, 'SC2': 0.191, 'SC3': 0.191},
 'POPC': {'NC3': 0.264,
  'PO4': 0.264,
  'GL1': 0.23,
  'GL2': 0.264,
  'C1A': 0.264,
  'D2A': 0.264,
  'C3A': 0.264,
  'C4A': 0.264,
  'C1B': 0.264,
  'C2B': 0.264,
  'C3B': 0.264,
  'C4B': 0.264},
 'POPE': {'NH3': 0.264,
  'PO4': 0.264,
  'GL1': 0.23,
  'GL2':